In [ ]:
import json
import pandas as pd
import requests
import sqlalchemy


def get_weathers(cities_df):

    API_key = ""
    
    city_list = cities_df["city_name"]

    # initializing emptly lists that will be filled with weather info per city and later be used to create a dataframe    countries = []
    cities= []
    times = []
    temps = []
    temps_feel = []
    temps_min = []
    temps_max = []
    humidities = []
    weather_mains = [] 
    weather_descriptions = []
    wind_speeds = []
    visabilities = []
    rain_probability = []
    city_ids = []

    # iterating through the list of cities to collect the weather data from
    for city in city_list:
        city_name = city

        # sending a GET request on the API call url
        weather = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?q={city_name}&appid={API_key}&units=metric")
        # using json()-method to access the pulled data in JSON format
        weather_json = weather.json()
     
        # going through the different levels and directories of the JSON-file to collect desired info, iterating over the cities
        for i in range(len(weather_json["list"])):
            countries.append(weather_json["city"]["country"])
            cities.append(weather_json["city"]["name"])
            times.append(weather_json["list"][i]["dt_txt"])
            temps.append(weather_json["list"][i]["main"]["temp"])
            temps_feel.append(weather_json["list"][i]["main"]["feels_like"])
            temps_min.append(weather_json["list"][i]["main"]["temp_min"])
            temps_max.append(weather_json["list"][i]["main"]["temp_max"])
            humidities.append(weather_json["list"][i]["main"]["humidity"])
            weather_mains.append(weather_json["list"][i]["weather"][0]["main"])
            weather_descriptions.append(weather_json["list"][i]["weather"][0]["description"])
            wind_speeds.append(weather_json["list"][i]["wind"]["speed"])
            visabilities.append(weather_json["list"][i]["visibility"])
            rain_probability.append(weather_json["list"][i]["pop"])
            city_ids.append(cities_df.loc[cities_df["city_name"]==city]["city_id"].item())

    # putting together the dataframe consisting of a dictionary with lists in it
    weathers_df = pd.DataFrame({"city_id": city_ids,
                            "forecast_time": times,
                            "outlook": weather_descriptions,
                            "temperature": temps,
                            "feels_like": temps_feel,
                            "wind_speed": wind_speeds,
                            "rain_prob": rain_probability
                            })
    # returning the dataframe
    return weathers_df
    
def lambda_handler(event, context):
    # setting variables for sql connection
    schema="gans"
    host=""
    user="admin"
    password=""
    port=3306
    con = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    
    # reading cities dataframe from sql database
    cities_sql = pd.read_sql_table("cities", con=con)
    print(cities_sql)
    # executing opeanweather API call function
    get_weathers(cities_sql).to_sql("weathers", if_exists="append", con=con, index=False)
    
    return {
    'statusCode': 200,
    'body': json.dumps('Hello from Lambda!')}